In [15]:
import pandas as pd
import numpy as np
import glob
from pathlib import Path

models = ['answer_gpt4', 'answer_gpt35', 'answer_bard', 'answer_claude', 'answer_vicuna-13b']
reviewers = ['gpt-4', 'claude-1', 'vicuna-13b', 'gpt-3.5-turbo-0301', 'text-bison@001']

def review_filename(modelA, modelB, reviewer):
    name = f'{modelA}-vs-{modelB}-{reviewer}-reviewer*.jsonl'
    glob_path = './ratings-*/' + name
    globs = glob.glob(glob_path)
    return globs[0]

def format_df(df):
    # map A wins to -1, B wins to 1, and ties to 0
    df.score = df.score.map({1: -1, 2: 1, 3: 0})
    df.sort_values(by='question_id', inplace=True)

def load_reviews(model, reviewers):
    dfs_list = []
    for modelA in models:
        for modelB in models:
            if modelA == modelB:
                continue

            scores = []
            for reviewer in reviewers:
                filename = review_filename(modelA, modelB, reviewer)
                # get df and add to array
                df = pd.read_json(filename, lines=True)[['question_id', 'score']]
                format_df(df)
                df['model_a'] = modelA
                df['model_b'] = modelB
                df['reviewer'] = reviewer
                dfs_list.append(df)
        
    # combine all dfs and shuffle
    reviews = pd.concat(dfs_list)
    return reviews

In [23]:
def majority_vote(df):
    def take_majority(frame):
        tolerance = 0.1
        x = frame.mean()
        if x > tolerance:
            return 1
        elif x < -tolerance:
            return -1
        else:
            return 0

    return df.groupby(['question_id', 'model_a', 'model_b'], as_index=False).agg({'score': take_majority})

In [25]:
from collections import defaultdict

# function obtained from https://colab.research.google.com/drive/1lAQ9cKVErXI1rEYq7hTKNaCQ5Q8TzrI5
def compute_elo(battles, K=32, SCALE=400, BASE=10, INIT_RATING=1000):
    rating = defaultdict(lambda: INIT_RATING)

    i = 0
    for rd, model_a, model_b, winner in battles[['model_a', 'model_b', 'score']].itertuples():
        i += 1
        ra = rating[model_a]
        rb = rating[model_b]
        ea = 1 / (1 + BASE ** ((rb - ra) / SCALE))
        eb = 1 / (1 + BASE ** ((ra - rb) / SCALE))
        if winner == -1:
            sa = 1
        elif winner == 1:
            sa = 0
        elif winner == 0:
            sa = 0.5
        else:
            print("problem @", model_a, model_b)
            raise Exception(f"unexpected vote {winner}")
        rating[model_a] += K * (sa - ea)
        rating[model_b] += K * (1 - sa - eb)

    print("Iterations: ", i)
    return rating


In [19]:
reviews = load_reviews(models, reviewers)

In [28]:
maj = majority_vote(reviews)
print(type(maj))
print(maj)
compute_elo(maj)

<class 'pandas.core.frame.DataFrame'>
      question_id            model_a            model_b  score
0               1        answer_bard      answer_claude      1
1               1        answer_bard       answer_gpt35     -1
2               1        answer_bard        answer_gpt4      1
3               1        answer_bard  answer_vicuna-13b      1
4               1      answer_claude        answer_bard     -1
...           ...                ...                ...    ...
1595           80        answer_gpt4  answer_vicuna-13b     -1
1596           80  answer_vicuna-13b        answer_bard      1
1597           80  answer_vicuna-13b      answer_claude      1
1598           80  answer_vicuna-13b       answer_gpt35      1
1599           80  answer_vicuna-13b        answer_gpt4      1

[1600 rows x 4 columns]
Iterations:  1600


defaultdict(<function __main__.compute_elo.<locals>.<lambda>()>,
            {'answer_bard': 828.7743322009458,
             'answer_claude': 1040.8108269125312,
             'answer_gpt35': 954.9558538386063,
             'answer_gpt4': 1378.09493913317,
             'answer_vicuna-13b': 797.3640479147488})